In [1]:
from pyspark.sql import SparkSession
import time

# 初始化 Spark 会话
spark = SparkSession.builder.appName("PartitionImpactExperiment").master("spark://ecnu01:7077") \
   .config("spark.executor.memory", "14g") \
   .config("spark.executor.cores", "16") \
   .config("spark.executor.instances", "2") \
   .getOrCreate()

sc = spark.sparkContext

# 定义不同的数据规模
data_sizes = [10 ** 7, 10 ** 6, 10 ** 5]

# 定义不同的分区数进行实验
partition_list = [256, 128, 64, 16, 4]

for data_size in data_sizes:
    print(f"\nTesting with data size of {data_size}")
    # 创建对应规模的数据集
    data = [(i % 1000, i) for i in range(data_size)]
    rdd = sc.parallelize(data)
    for num_partitions in partition_list:
        total_time = 0
        print(f"Testing with {num_partitions} partitions")
        # 对每个分区数进行5次实验
        i = 0
        for i in range(5):
            # 调整分区数
            rdd_repartitioned = rdd.repartition(num_partitions)  # 重新分区

            # 记录开始时间
            start_time = time.time()

            # 执行宽依赖操作（reduceByKey 会触发 Shuffle）
            result = rdd_repartitioned.reduceByKey(lambda x, y: x + y).count()

            # 记录结束时间
            end_time = time.time()

            temp = end_time - start_time
            total_time += temp
            print(f"第 {i+1} 次：", temp)

        # 计算平均时间
        avg_time = total_time / 5
        print(f"Average Execution time with {num_partitions} partitions: {avg_time:.2f} seconds")
        print(f"Result count: {result}")

# 停止 Spark 上下文
sc.stop()


Testing with data size of 10000000
Testing with 256 partitions
第 1 次： 8.011298179626465
第 2 次： 3.2403860092163086
第 3 次： 3.1574063301086426
第 4 次： 3.000058889389038
第 5 次： 3.156585693359375
Average Execution time with 256 partitions: 4.11 seconds
Result count: 1000
Testing with 128 partitions
第 1 次： 3.9466512203216553
第 2 次： 3.2779223918914795
第 3 次： 2.9741179943084717
第 4 次： 3.4183452129364014
第 5 次： 2.9113047122955322
Average Execution time with 128 partitions: 3.31 seconds
Result count: 1000
Testing with 64 partitions
第 1 次： 2.170232057571411
第 2 次： 2.130234479904175
第 3 次： 2.1004676818847656
第 4 次： 2.2368946075439453
第 5 次： 2.197067975997925
Average Execution time with 64 partitions: 2.17 seconds
Result count: 1000
Testing with 16 partitions
第 1 次： 1.7939014434814453
第 2 次： 1.6877632141113281
第 3 次： 1.7893381118774414
第 4 次： 1.7415621280670166
第 5 次： 1.8409254550933838
Average Execution time with 16 partitions: 1.77 seconds
Result count: 1000
Testing with 4 partitions
第 1 次： 2.912